# Diffusion data processing using FSL

![flowchart](dwi_flowchart.png)

### Commands for pre-pre-processing
- convert from dicoms to nifti/bval/bvec
- extract B0 volume of the nifti file (assuming it's the first volume)
- skullstrip the B0 volume

In [ ]:
%%bash
cd  DTIprocessing/Subj1
dcm2nii DICOM_folder
mv DICOM_folder/*.nii.gz Subj1.nii.gz
mv DICOM_folder/*.bvec bvec.bvec
mv DICOM_folder/*.bval bval.bval
fslroi Subj1.nii.gz 0 b0.nii.gz 128 0 128 0 60 0 1
bet b0.nii.gz nodif_brain –f 0.4 –g 0 –m

### Commands for preprocessing 

#### Case 1
If you have only one dti data per subject AND no field mapping data collected, then the only preprocessing that can be done is eddy current correction

In [ ]:
%%bash
eddy_correct Subj1.nii.gz  data 0

If you are using FSL 5.0.11

In [ ]:
%%bash
eddy --imain=Sub1.nii.gz --mask=nodif_brain_mask.nii.gz --acqp=acqparams.txt --index=index.txt --bvecs=bvec.bvec --bvals=bval.bval --topup=my_topup_results --niter=8 --fwhm=10,8,4,2,0,0,0,0 --repol --out=data --mporder=6 --slspec=my_slspec.txt --s2v_niter=5 --s2v_lambda=1 --s2v_interp=trilinear


where acqparams.txt is a textfile with entries                                                                          0 1 0 1                                                                                                               in a single row. You need to create this! Similarly, index.txt is a text file with as many 1's as the # volumes in your Sub1.nii.gz. The 1's can be entered as columns or rows.

Afterwards, you are ready for fitting tensor model:


In [ ]:
%%bash
dtifit –k data.nii.gz –o dti –m nodif_brain_mask.nii.gz –r bvec.bvec –b bval.bval

#### Case 2
If you have two sets of data collected with phase encoding polarity switched, then preprocessing for geometric distortion correction can be done prior to eddy current correction. For older FSL versions (<5.0.11?).

In [ ]:
%%bash
cd  DTIprocessing/Subj1
dcm2nii DICOM_folder1
dcm2nii DICOM_folder2

mv DICOM_folder1/*.nii.gz Subj1_A2P.nii.gz
mv DICOM_folder2/*.nii.gz Subj1_P2A.nii.gz

fslroi Subj1_A2P.nii.gz b0_A2P.nii.gz 0 128 0 128 0 60 0 1 # the numbers are the dimensions of the data
fslroi Subj1_P2A.nii.gz b0_P2A.nii.gz 0 128 0 128 0 60 0 1

bet b0_A2P.nii.gz nodif_brain –f 0.4 –g 0 –m

fslmerge -t A2P_P2A_b0 b0_A2P.nii.gz b0_P2A.nii.gz 

topup --imain=A2P_P2A_b0 --datain=acqparams.txt --config=b02b0.cnf --out=topup_results --iout=my_hifi_b0

applytopup --imain=A2P,P2A --inindex=1,2 --datatin=acqparams.txt --topup=topup_results --out=my_hifi_images

eddy_correct my_hifi_images.nii.gz data 0

dtifit –k data.nii.gz –o dti –m nodif_brain_mask.nii.gz –r bvec.bvec –b bval.bval

For new FSL version (>5.0.11):

In [ ]:
%%bash
eddy --imain=Sub1.nii.gz --mask=nodif_brain_mask.nii.gz --acqp=acqparams.txt --index=index.txt --bvecs=bvec.bvec --bvals=bval.bval --topup=my_topup_results --niter=8 --fwhm=10,8,4,2,0,0,0,0 --repol --out=data --mporder=6 --slspec=my_slspec.txt --s2v_niter=5 --s2v_lambda=1 --s2v_interp=trilinear
dtifit –k data.nii.gz –o dti –m nodif_brain_mask.nii.gz –r bvec.bvec –b bval.bval

where acqparams.txt is a textfile with entries
0 1 0 1
0 -1 0 1 
in two rows. 
You need to create this! Similarly, index.txt is a text file with as many 1s as the # volumes in your Sub1_A2P.nii.gz and as many 2s as the # volumes in Sub1_P2A.nii.gz 

Check the output of dtifit and make sure the diffusion directions are estimated correctly!!

### Other post-processing options
At this point, you may decide to work with the scalar maps and do a statistical processing of that. To bring all subjects to a common space, you may also run a registration using FLIRT or FNIRT.

For whole brain voxel-wise statistical analysis, you may use feat’s GLM, or randomize or say take it to SPM, AFNI, R and perform analysis there.

 For ROI-based analysis, you can extract mean FA values from say roixyz.nii.gz

fslmerge
fslstats 

### Fiber Tracking Using FSL

After dtifit has been successfully run and the fiber directions checked for accuracy, run:

In [ ]:
%%bash
bedpostx  Subj1 –n 3

(three fiber per voxel)
can also work on multi-shell data.

	Come back after 24 hours and we are ready for fiber tracking!!